# Compute efficiency distributions for a proton sample

In [4]:
#!python3 -m pip install --user uproot4 awkward1 mplhep

In [10]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import mplhep

### Proton reconstruction flavours

We will look at two ways to reconstruct protons in PPS: _single-RP_ and _multi-RP_

The flag below controls which proton reconstruction is used.

In the multi-RP reconstruction, both "near" and "far" tracking detector stations in one spectrometer arm are used. 

In the single-RP reconstruction, the proton kinematics is only partially reconstructed from a single detector station. In this case we will use the "far" station. In the 2017 detector configuration, this corresponds to the stations using pixel detectors.

In [11]:
fileName_data = "data/output-UL2017B-PreSel.h5"
#fileName_data = "data/output-UL2017C1-PreSel.h5"
#fileName_data = "data/output-UL2017F1-PreSel.h5"
#fileNames_data = [
#    'data/output-UL2017B-PreSel.h5',
#    'data/output-UL2017C1-PreSel.h5',
#    'data/output-UL2017F1-PreSel.h5'
#]

#proton_selection = "SingleRP"
proton_selection = "MultiRP"

## Access the data

In [13]:
df_signal = None
event_counts_signal = None
selections_str_signal = None

with h5py.File( fileName_signal, 'r' ) as f:
    print ( list(f.keys()) )
    dset = f['protons']
    print ( dset.shape )
    print ( dset[:,:] )
    dset_columns = f['columns']
    print ( dset_columns.shape )
    columns = list( dset_columns )
    print ( columns )
    columns_str = [ item.decode("utf-8") for item in columns ]
    print ( columns_str )
    
    dset_counts = f['event_counts']
    event_counts_signal = list( dset_counts )
    print ( event_counts_signal )
    
    dset_selections = f['selections']
    selections_ = list( dset_selections )
    print ( selections_ )
    selections_str_signal = [ item.decode("utf-8") for item in selections_ ]
    print ( selections_str_signal )
    
    df_signal = pd.DataFrame( dset, columns=columns_str )
    
    df_signal = df_signal[ ['Run', 'LumiSection', 'EventNum', 'CrossingAngle', 
                            'MultiRP', 'Arm', 'RPId1', 'RPId2', 'TrackX1', 'TrackY1', 'TrackX2', 'TrackY2',
                            'Xi', 'T', 'ThX', 'ThY',
                            'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMuPlus', 'XiMuMuMinus'] ].astype( { "Run": "int64", "LumiSection": "int64", "EventNum": "int64", "MultiRP": "int32", "Arm": "int32", "RPId1": "int32", "RPId2": "int32", "ExtraPfCands": "int32" } )
    

['columns', 'event_counts', 'protons', 'selections']
(212317, 38)
[[1.00000000e+00 1.00000000e+00 4.29496730e+09 ... 9.13764843e-04
  2.35298625e-02 4.41197067e-03]
 [1.00000000e+00 1.00000000e+00 4.29496730e+09 ... 8.79983866e-04
  4.58796041e-03 8.15103097e-02]
 [1.00000000e+00 1.00000000e+00 4.29496730e+09 ... 8.79983866e-04
  4.58796041e-03 8.15103097e-02]
 ...
 [1.00000000e+00 7.56000000e+02 4.29496730e+09 ... 1.85212595e-04
  3.62914948e-02 3.05966531e-03]
 [1.00000000e+00 7.56000000e+02 4.29496730e+09 ... 1.85212595e-04
  3.62914948e-02 3.05966531e-03]
 [1.00000000e+00 7.56000000e+02 4.29496730e+09 ... 1.50995244e-04
  2.04409396e-02 4.87857127e-03]]
(38,)
[b'Run', b'LumiSection', b'BX', b'EventNum', b'CrossingAngle', b'MultiRP', b'Arm', b'RPId1', b'RPId2', b'TrackX1', b'TrackY1', b'TrackX2', b'TrackY2', b'Xi', b'T', b'ThX', b'ThY', b'Time', b'TrackThX_SingleRP', b'TrackThY_SingleRP', b'Track1ThX_MultiRP', b'Track1ThY_MultiRP', b'Track2ThX_MultiRP', b'Track2ThY_MultiRP', b'Muon0

### Proton efficiency evaluation

We will evaluate the efficiency correction for _multi-RP_ protons. 

This has to be computed as the product of the efficiency of the "near" detector station (silicon strips, in 2017) and the "multiRP efficiency".

The efficiency components for the near RP station is divided in two parts: multitrack and sensor. 
The former accounts the inefficiency caused by strips detectors not being abla to reconstruct multiple tracks in the same event. This is a single numeric factor.
The sensor efficiency takes into account local inefficiencies caused, e.g., by radiation damage and is supplied as a 2D, xy map. 

The multiRP efficiency accounts instead for both the sensor efficiency of the "far" RP (pixels in 2017 and 2018) and the inefficiency component given by the proton chance of interacting while travelling from the near to the far RP. Again, this is supplied as a 2D, xy map.

A different measurement is available for each data period here included: 2017B, 2017C1, 2017F1.

Despite being counter-intuitive, it should be noted that, for near RP efficiencies one should use the xy coordinates measured in the far RP (TrackX2/TrackY2) and for multiRP the ones measured in the near RP (TrackX1/TrackY1). This is a consequence of the Tag&Probe method used to produce the efficiency corrections.

In [45]:
import ROOT

# Efficiency correction files
strips_efficiency_file = ROOT.TFile.Open("/eos/project/c/ctpps/subsystems/Strips/StripsTracking/PreliminaryEfficiencies_July132020_1D2DMultiTrack.root")
multiRP_efficiency_file = ROOT.TFile.Open("/eos/project/c/ctpps/subsystems/Pixel/RPixTracking/pixelEfficiencies_multiRP.root")

data_taking_period = "2017B"
year = data_taking_period[:4]

strips_multitrack_efficiency = {}
strips_sensor_efficiency = {}
multiRP_efficiency = {}

# Retrieve histograms from files and save them in dictionaries for future usage
for sector in ["45","56"]:
    strips_multitrack_efficiency[sector] = strips_efficiency_file.Get("Strips/2017/2017B/h45multitrackeff_2017B_avg_RP3").GetBinContent(1)
    strips_sensor_efficiency[sector] = strips_efficiency_file.Get("Strips/"+year+"/"+data_taking_period+"/h"+sector+"_"+data_taking_period+"_all_2D")
    multiRP_efficiency[sector] = multiRP_efficiency_file.Get("Pixel/"+year+"/"+data_taking_period+"/h"+sector+"_220_"+data_taking_period+"_all_2D")

0.4166666567325592
0.6923076923076923
